In [1]:
from typing import List
import copy

In [2]:
DUMMY_STARTING_POS = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""

## Part 1

In [3]:
def test_count_positions(func):
    expected_output = 41

    function_output = func(DUMMY_STARTING_POS)

    if function_output != expected_output:
        raise ValueError("function does not return correct value")
    else:
        print("passed")

In [4]:
class Guard:
    def __init__(self, starting_map: str, loggers: bool):
        self.current_map = self.convert_map_to_list(starting_map)
        self.loggers = loggers
        self.in_area = True
    
    def convert_map_to_list(self, map_str):
        """
        Converts a map string into a list of lists. Each inner list represents a row.
        """
        if isinstance(map_str, list):
            return map_str
        map_list = map_str.split("\n")
        map_list = [list(row) for row in map_list]

        return map_list
    
    def find_current_position(self) -> tuple:
        """
        Finds the curent coordinates of the guard.
        """
        for row_idx, row in enumerate(self.current_map):
            if "^" in row:
                return row.index("^"), row_idx
    
    def rotate_map(self):
        """
        Roates the map 90 degrees counter clockwise
        """
        num_rows = len(self.current_map)
        new_map = []
        for i in range(num_rows):
            new_map.append([row[num_rows - 1 - i] for row in self.current_map])
        
        self.current_map = new_map

    def move(self):
        """
        Moves the guard one space forward if there is no obstacle, or rotates the map if there is an obstacle.
        """
        if not self.in_area:
            if self.loggers:
                print("Guard is not in the area")
            return
        
        col_num, row_num = self.find_current_position()
        
        if row_num == 0:
            self.current_map[row_num][col_num] = "X"
            self.in_area = False
            logger = "Guard has now left the area"

        elif self.current_map[row_num - 1][col_num] == "#":
            self.rotate_map()
            logger = "Guard has turned 90 degrees"

        else:
            self.current_map[row_num][col_num] = "X"
            self.current_map[row_num - 1][col_num] = "^"
            logger = "Guard has taken a step"
        
        if self.loggers:
            print(logger)


In [5]:
def count_positions(starting_pos: str) -> int:
    """
    Simulates the guard's path and counts all positions visited.
    """
    guard = Guard(starting_pos, loggers=False)
    while guard.in_area:
        guard.move()
    
    count = 0
    for rows in guard.current_map:
        count += rows.count("X")

    return count

In [6]:
test_count_positions(count_positions)

passed


In [7]:
with open('input_6.txt') as file:
    starting_position = file.read()

In [8]:
count_positions(starting_position)

5177

## Part 2

In [9]:
def test_count_loop_possibilities(func):
    expected_output = 6

    function_output = func(DUMMY_STARTING_POS)

    if function_output != expected_output:
        raise ValueError("function does not return correct value")
    else:
        print("passed")

In [10]:
class AdvancedGuard(Guard):
    """
    All the functionality to the ordinary guard class from above with additional features:
        - Number of map rotations is saved so that orientation can be derived.
        - Each time an obstacle is hit, the coordinates of the guard and the orientation of the map is saved.
        - The in_loop variable is set to true when the same obstacle is hit twice on the same orientatiom, as this means that the guard is stuck in a loop.
    """
    def __init__(self, starting_map: str, loggers: bool):
        self.current_map = self.convert_map_to_list(starting_map)
        self.loggers = loggers
        self.rotations_count = 0
        self.obstacles_hit_coordinates = []
        self.in_area = True
        self.in_loop = False

        col_num, row_num = self.find_current_position()
        self.current_coordinates = (col_num, row_num, 0)       
    
    def move(self):
        """
        Moves the guard one space forward if there is no obstacle, or rotates the map if there is an obstacle.
        """
        if not self.in_area:
            if self.loggers:
                print("Guard is not in the area")
            return
    
        col_num, row_num, orientation = self.current_coordinates

        if row_num == 0:
            self.current_map[row_num][col_num] = "X"
            self.in_area = False
            self.current_coordinates = (None, None, orientation)
            if self.loggers:
                print("Guard has now left the area")
            return

        elif self.current_map[row_num - 1][col_num] == "#":
            if self.current_coordinates in self.obstacles_hit_coordinates:
                self.in_loop = True
            else:
                self.obstacles_hit_coordinates.append(self.current_coordinates)

            self.rotate_map()
            self.rotations_count += 1
            logger = f"Guard has turned 90 degrees. Loop status: {self.in_loop}"
            
        else:
            self.current_map[row_num][col_num] = "X"
            self.current_map[row_num - 1][col_num] = "^"
            logger = "Guard has taken a step"
        
        col_num, row_num = self.find_current_position()
        orientation =  self.rotations_count % 4
        self.current_coordinates = (col_num, row_num, orientation)
        
        if self.loggers:
            print(logger)


In [11]:
def test_map_for_loops(map: List[list], row: int, col: int) -> bool:
    """
    Returns True if placing an obstacle in a position would create a loop, and False otherwise.
    """
    new_map = copy.deepcopy(map)
    new_map[row][col] = "#"

    new_guard = AdvancedGuard(new_map, loggers=False)
    while new_guard.in_area and not new_guard.in_loop:
        new_guard.move()
    
    return new_guard.in_loop


In [12]:
def count_loop_possibilities(starting_pos: str) -> int:
    """
    Simulates the guard's path and at each point places an obstacle and resimulates to check for loops.
    """
    guard = AdvancedGuard(starting_pos, loggers=False)

    loop_possibilities = 0
    steps = 0
    while guard.in_area:
        col_num, row_num, orientation = guard.current_coordinates
        map = guard.current_map 
        if map[row_num-1][col_num] not in ("X", "#"):
            loop = test_map_for_loops(map, row_num-1, col_num)
            if loop:
                loop_possibilities += 1
        guard.move()
        steps += 1

    return loop_possibilities


In [13]:
count_loop_possibilities(DUMMY_STARTING_POS)

6

In [14]:
test_count_loop_possibilities(count_loop_possibilities)

passed


In [15]:
count_loop_possibilities(starting_position)

1686